In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import pandas as pd


In [3]:
# Load Dataset
file_path = "Personalized_Diet_Recommendations.csv"  # Modify path if needed
df = pd.read_csv("/content/Personalized_Diet_Recommendations.csv")
df = df.dropna()
print(df.head())  # Check if data is loaded correctly

   Patient_ID  Age  Gender  Height_cm  Weight_kg    BMI Chronic_Disease  \
13     P00014   28   Other        164         93  34.58        Diabetes   
18     P00019   41   Other        183         96  28.67        Diabetes   
32     P00033   75  Female        154         54  22.77         Obesity   
37     P00038   66  Female        154         59  24.88        Diabetes   
42     P00043   77  Female        167        104  37.29    Hypertension   

    Blood_Pressure_Systolic  Blood_Pressure_Diastolic  Cholesterol_Level  ...  \
13                      104                       100                187  ...   
18                      119                        75                203  ...   
32                      135                       117                165  ...   
37                      139                       104                287  ...   
42                       96                        95                207  ...   

    Protein_Intake Carbohydrate_Intake Fat_Intake  Preferred_C

In [4]:
# Convert structured data into textual descriptions for BERT
def create_textual_features(row):
    return (
        f"Patient is a {row['Age']}-year-old {row['Gender']} with BMI of {row['BMI']}. "
        f"Chronic disease: {row['Chronic_Disease']}. Blood Pressure: {row['Blood_Pressure_Systolic']}/"
        f"{row['Blood_Pressure_Diastolic']}. Cholesterol: {row['Cholesterol_Level']}. "
        f"Blood Sugar: {row['Blood_Sugar_Level']}. Genetic Risk Factor: {row['Genetic_Risk_Factor']}. "
        f"Dietary Habits: {row['Dietary_Habits']}. Preferred Cuisine: {row['Preferred_Cuisine']}. "
        f"Food Aversions: {row['Food_Aversions']}. Protein Intake: {row['Protein_Intake']}g, "
        f"Carbs Intake: {row['Carbohydrate_Intake']}g, Fat Intake: {row['Fat_Intake']}g."
    )

In [5]:
# Apply transformation
df["user_text"] = df.apply(create_textual_features, axis=1)

In [6]:
# Tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [16]:
# Label Encoding
df["label"] = df["Recommended_Meal_Plan"].astype("category").cat.codes
label_mapping = dict(enumerate(df["Recommended_Meal_Plan"].astype("category").cat.categories))

In [17]:
# Dataset Class
class DietDataset(Dataset):
    def __init__(self, data):
        self.data = data.reset_index(drop=True)  # Reset index for stability

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data["user_text"][idx]
        tokens = tokenizer(
            text,
            padding="max_length",
            truncation=True,
            max_length=128,
            return_tensors="pt"
        )

        return {
            "input_ids": tokens["input_ids"].squeeze(0),
            "attention_mask": tokens["attention_mask"].squeeze(0),
            "label": torch.tensor(self.data["label"].iloc[idx], dtype=torch.long)
        }


In [18]:
# Split Data
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)
train_dataset = DietDataset(train_data)
test_dataset = DietDataset(test_data)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

In [19]:
# Define BERT-based Model
class BertDietRecommender(nn.Module):
    def __init__(self, num_classes):
        super(BertDietRecommender, self).__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        self.fc = nn.Linear(768, num_classes)  # Output classes = unique meal plans

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        logits = self.fc(outputs.pooler_output)  # Use the [CLS] token embedding
        return logits

In [20]:
# Initialize Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = len(df["Recommended_Meal_Plan"].unique())  # Unique meal plan categories
model = BertDietRecommender(num_classes).to(device)


In [22]:
# Save the model after training
torch.save(model.state_dict(), "bert_diet_recommender.pth")


In [23]:
# Load or Train Model
model_path = "bert_diet_recommender.pth"
if torch.cuda.is_available():
    model.load_state_dict(torch.load(model_path, map_location=device))
else:
    model.load_state_dict(torch.load(model_path, map_location=torch.device("cpu")))

In [24]:
# Predict Diet Recommendation
def recommend_diet(user_input):
    model.eval()
    tokens = tokenizer(user_input, return_tensors="pt", padding=True, truncation=True, max_length=128)
    input_ids = tokens["input_ids"].to(device)
    attention_mask = tokens["attention_mask"].to(device)

    with torch.no_grad():
        logits = model(input_ids, attention_mask)
        predicted_index = torch.argmax(logits, dim=1).item()

    return label_mapping[predicted_index]

In [25]:
def get_user_input():
    print("Please enter your health details to get a personalized diet recommendation.")

    age = input("Enter your age: ")
    gender = input("Enter your gender (Male/Female): ")
    bmi = input("Enter your BMI: ")
    chronic_disease = input("Do you have any chronic disease? (Yes/No, specify if Yes): ")
    blood_pressure = input("Enter your blood pressure (Systolic/Diastolic): ")
    cholesterol = input("Enter your cholesterol level (Low/Normal/High): ")
    blood_sugar = input("Enter your blood sugar level (Normal/High): ")
    genetic_risk = input("Do you have any genetic risk factors? (Yes/No): ")
    diet_habits = input("Describe your dietary habits (Veg, Non-Veg, Vegan, etc.): ")
    cuisine = input("What is your preferred cuisine? (e.g., Mediterranean, Indian, Asian, etc.): ")
    food_aversions = input("Do you have any food allergies/aversions? (List them or say None): ")
    protein = input("Enter your daily protein intake (grams): ")
    carbs = input("Enter your daily carbohydrate intake (grams): ")
    fats = input("Enter your daily fat intake (grams): ")

    user_text = (
        f"Patient is a {age}-year-old {gender} with BMI of {bmi}. "
        f"Chronic disease: {chronic_disease}. Blood Pressure: {blood_pressure}. "
        f"Cholesterol: {cholesterol}. Blood Sugar: {blood_sugar}. Genetic Risk Factor: {genetic_risk}. "
        f"Dietary Habits: {diet_habits}. Preferred Cuisine: {cuisine}. "
        f"Food Aversions: {food_aversions}. Protein Intake: {protein}g, "
        f"Carbs Intake: {carbs}g, Fat Intake: {fats}g."
    )

    return user_text

In [26]:
# Get user details & Predict Recommended Diet Plan
user_input = get_user_input()
recommended_diet = recommend_diet(user_input)

print(f"\n🔹 Recommended Meal Plan: {recommended_diet} ")

Please enter your health details to get a personalized diet recommendation.
Enter your age: 19
Enter your gender (Male/Female): 30.5
Enter your BMI: 30.5
Do you have any chronic disease? (Yes/No, specify if Yes): No
Enter your blood pressure (Systolic/Diastolic): 140/70
Enter your cholesterol level (Low/Normal/High): High
Enter your blood sugar level (Normal/High): High
Do you have any genetic risk factors? (Yes/No): No
Describe your dietary habits (Veg, Non-Veg, Vegan, etc.): Veg
What is your preferred cuisine? (e.g., Mediterranean, Indian, Asian, etc.): Asian
Do you have any food allergies/aversions? (List them or say None): Nut allergy
Enter your daily protein intake (grams): 150
Enter your daily carbohydrate intake (grams): 100
Enter your daily fat intake (grams): 90

🔹 Recommended Meal Plan: Balanced Diet 
